In [3]:
from datetime import datetime
filename = datetime.now().strftime('%d-%m-%y-%H_%M_ckpt.pth')
filename

'29-01-26-20_39_ckpt.pth'

In [5]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

checkpoint = "Langboat/mengzi-t5-base"
model = T5ForConditionalGeneration.from_pretrained(checkpoint)
tokenizer = T5Tokenizer.from_pretrained(checkpoint) 

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [3]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [4]:
tokenizer.tokenize("阿斯顿撒啊阿德")

['阿', '斯顿', '撒', '啊', '阿', '德']

In [25]:
ids = tokenizer.encode("阿斯顿撒啊阿德", max_length=10, padding="max_length")
ids

[633, 20982, 3471, 196, 633, 273, 1, 0, 0, 0]

In [27]:
tokenizer.decode(ids, skip_special_tokens=True)

'阿斯顿撒啊阿德'

In [28]:
tokenizer.decode(ids)

'阿斯顿撒啊阿德</s><pad><pad><pad>'

In [49]:
data = tokenizer(["阿斯顿撒啊阿德", "范文芳大"], max_length=10, padding="max_length", truncation=True)
data

{'input_ids': [[633, 20982, 3471, 196, 633, 273, 1, 0, 0, 0], [2860, 180, 3100, 28, 1, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 0, 0, 0], [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]]}

In [ ]:
from utils import collote_fn

model(**collote_fn(data))

In [2]:
import evaluate 

evaluate.load('bleu')

c:\Users\lamye\miniconda3\envs\transformers\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


EvaluationModule(name: "bleu", module_type: "metric", features: [{'predictions': Value('string'), 'references': List(Value('string'))}, {'predictions': Value('string'), 'references': Value('string')}], usage: """
Computes BLEU score of translated segments against one or more references.
Args:
    predictions: list of translations to score.
    references: list of lists of or just a list of references for each translation.
    tokenizer : approach used for tokenizing `predictions` and `references`.
        The default tokenizer is `tokenizer_13a`, a minimal tokenization approach that is equivalent to `mteval-v13a`, used by WMT.
        This can be replaced by any function that takes a string as input and returns a list of tokens as output.
    max_order: Maximum n-gram order to use when computing BLEU score.
    smooth: Whether or not to apply Lin et al. 2004 smoothing.
Returns:
    'bleu': bleu score,
    'precisions': geometric mean of n-gram precisions,
    'brevity_penalty': brevity

In [48]:
predictions = ["hello there general kenobi", "foo bar foobar"]
references = [
         ["hello there kenobi", "hello there!"],
         ["foo bar foobar"]
     ]
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predictions, references=references)
results

{'bleu': 0.0,
 'precisions': [0.8571428571428571, 0.6, 0.3333333333333333, 0.0],
 'brevity_penalty': 1.0,
 'length_ratio': 1.1666666666666667,
 'translation_length': 7,
 'reference_length': 6}

In [22]:
input_ids = tokenizer(
   ["弄好", "解决"],
   padding=True,   
   truncation=True, 
   return_tensors="pt"
).input_ids  # Batch size 1
outputs = model.generate(input_ids)
print(outputs)
print(tokenizer.decode(outputs[0]))
tokenizer.decode(outputs[0], skip_special_tokens=True)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


tensor([[    0, 32127, 21957, 32126,     1],
        [    0, 32127,     1,     0,     0]])
<pad> <extra_id_0> 解决这个问题 <extra_id_1> </s>


'解决这个问题'